<a href="https://colab.research.google.com/github/djmalice/GenAInotebooks/blob/main/Multi_agent_system_using_crew_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ["OPENAI_MODEL_NAME"] = (
    "gpt-4o"  # we need a larger context window for this project
)

Install requirements

In [ ]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.12.5
!pip install openai
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.5.4 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.
embedchain 0.1.102 requires pypdf<4.0.0,>=3.11.0, but you have pypdf 4.2.0 which is incompatible.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parse PDF files for home disclosures and store in txt to be sent by agent to Chatgpt



In [ ]:
from pathlib import Path
from openai import OpenAI
from google.colab import drive

path = Path('/content/drive/My Drive')

from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

filename = path/"WALLACE.pdf"

# Extracts the elements from the PDF
elements = partition_pdf(
    filename=filename,

    # Unstructured Helpers
    strategy="fast",
    infer_table_structure=True,
    model_name="yolox"
)
content = "\n".join([element.text for element in elements])


In [ ]:
with open("pest.txt", "w", encoding="utf-8") as transcript_file:
    transcript_file.write(content)

In [ ]:
filename = path/"AVID.pdf"

# Extracts the elements from the PDF
elements = partition_pdf(
    filename=filename,

    # Unstructured Helpers
    strategy="fast",
    infer_table_structure=True,
    model_name="yolox"
)
content = "\n".join([element.text for element in elements])

with open("avid.txt", "w", encoding="utf-8") as transcript_file:
    transcript_file.write(content)

# Install Crew AI and tools

In [ ]:
# Install the mains crewAI package
!pip install crewai

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
!pip install 'crewai[tools]'

  Using cached pypdf-3.17.4-py3-none-any.whl (278 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 4.2.0
    Uninstalling pypdf-4.2.0:
      Successfully uninstalled pypdf-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.22.0 requires pypdf>=4.0, but you have pypdf 3.17.4 which is incompatible.


# Create crew agents. One agent each to review document type. Additional agent to compute cost of repairs.

In [ ]:
from logging import RootLogger
from crewai import Agent, Crew
from crewai_tools import FileReadTool
from langchain_openai import OpenAI
from google.colab import userdata

# Initialize LLM
llm = OpenAI(api_key=userdata.get('OPENAI_API_KEY'), temperature=0.7)

inspection_reviewer = Agent(
    role="Real Estate Buyer's agent",
    goal="Highlight all the problems in the pest inspection report",
    backstory="""You are an agent in Real Estate helping buyers reading disclosures. Read through the
                  pest.txt and highlight all the issues that you can find in the report""",
    tools=[
        FileReadTool(),  # Tool to read the transcript file we generated above
    ],
    llm=llm,
    verbose=True,
)

seller_questionnaire_reviewer = Agent(
    role="Real Estate Buyer's agent",
    goal="Read the agent visual disclosure document and report the items that were problems to be fixed at home after doing home inspection",
    backstory="""You are an agent in Real Estate helping buyers reading disclosures. Read through the
                  avid.txt and highlight all items marked yes in the report""",
    tools=[
        FileReadTool(),  # Tool to read the transcript file we generated above
    ],
    llm=llm,
    verbose=True,
)

contractor = Agent(
    role="Real Estate Contractor",
    goal="For all the problematic items in pest inspection report and agent visual disclosure document, provide recommendations to fix these items with estimated cost per item and also the total cost of fixing these itmes",
    backstory=""" You are an excellent contractor who quotes reasonable prices fixes problems in home inspection reports and visual disclosure documents.""",
    llm=llm,
    verbose=True,
)

# Give all agents task and the contractor agent reads from previous 2 agents to provide cost of all activites.

In [ ]:
from crewai import Task

inspect_task = Task(
    description="Find all the issues in the pest inspection report",
    input_file="pest.txt",
    expected_output="A bullet list of all the problems and the recommendations to fix these problems",
    agent=inspection_reviewer,
    output_file="inspection.txt"
)
spq_task = Task(
    description="Find all the items that were problematic in the visual inspection.",
    input_file="avid.txt",
    expected_output="A bullet list of all the items that were problems to be fixed",
    agent=seller_questionnaire_reviewer,
    output_file="avid_output.txt"
)

contractor_task = Task(
    description="For all the problematic items in pest inspection report and agent visual disclosure document, provide recommendations to fix these items with estimated cost per item and also the total cost of fixing these itmes",
    expected_output="A table with 2 columns -> Item to be fixed and Cost. Finally the total cost",
    agent=contractor,
    context=[inspect_task,spq_task],
    output_file="cost.txt"
)

# Run agents

In [ ]:
crew = Crew(
    agents=[inspection_reviewer,seller_questionnaire_reviewer, contractor],
    tasks=[inspect_task, spq_task, contractor_task],
    verbose=True,
)
crew.kickoff()

 [DEBUG]: == Working Agent: Real Estate Buyer's agent
 [INFO]: == Starting Task: Find all the issues in the pest inspection report


> Entering new CrewAgentExecutor chain...

I need to read the pest inspection report and highlight all the issues in order to give a complete recommendation on how to fix them.

Action: Read a file's content
Action Input: {"file_path": "pest.txt"} 

1. Substructure Areas:
ITEM 1A
Pieces of cellulose debris (wood, paper products) were inspected in the subarea and no fungus infection was noted. This does not mean all the debris is not infected, as the inspector does not turn over each piece.
RECOMMENDATION Remove the cellulose debris of a rakable size from the subarea and dispose of. If while removing debris termites are found, a supplemental report will be issued with additional findings and ocsts. ****** This is a Section 2 Item ******
ITEM 1B
Others have made repairs to substructure framing in the past. No representations are made as to the presence or a

'| Item to be fixed | Cost |\n|------------------|------|\n| Debris           | $500 |\n| Further Inspection | $100 |\n| Fumigation | $5000 |\n| Repair Crack | $200 |\n| Replace Damaged Knob | $50 |\n| Repair Cracks and Secure Toilet | $300 |\n| Secure Toilet | $100 |\n\nTotal Cost: $6150'